In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision


import os
import sys
import time
import torch
import requests
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.datasets as datasets



# Constants
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
N_CLASSES = 10
BATCH_SIZE = 32
CUDA = torch.cuda.is_available()

TRAIN_FOLDER = r"data/train"
VALID_FOLDER = r"data/validation"
TEST_FOLDER = r"data/test"
RESULT_FOLDER = r"results"


In [6]:
  #Init folder data (NPYs)
  
  urls = [
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/key.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/ladder.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/tennis%20racquet.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pizza.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/stop%20sign.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/wheel.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/fork.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy',
        'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy',
    ]
  
  class_name = ['apple', 'banana', 'book', 'fork', 'key', 'ladder', 'pizza', 'stop_sign', 'tennis_racquet', 'wheel']
   
  def createDir(path):
    if not os.path.exists(path):
        os.makedirs(path)
    
  def gen_bar_updater(pbar):
    def bar_update(count, block_size, total_size):
        if pbar.total is None and total_size:
            pbar.total = total_size
        progress_bytes = count * block_size
        pbar.update(progress_bytes - pbar.n)
    return bar_update   
    
  def download_url(url, root, filename):
      from six.moves import urllib
      root = os.path.expanduser(root)
      fpath = os.path.join(root, filename + ".npy")

      createDir(root)

      # downloads file
      if os.path.isfile(fpath):
          a = 1
          #print('Using downloaded and verified file: ' + fpath)
      else:
          try:
              print('Downloading ' + url + ' to ' + fpath)
              urllib.request.urlretrieve(
                  url, fpath,
                  reporthook = gen_bar_updater(tqdm(unit='B', unit_scale=True))
              )
          except OSError:
              if url[:5] == 'https':
                  url = url.replace('https:', 'http:')
                  print('Failed download. Trying https -> http instead.'
                        ' Downloading ' + url + ' to ' + fpath)
                  urllib.request.urlretrieve(
                      url, fpath,
                      reporthook = gen_bar_updater(tqdm(unit='B', unit_scale=True))
                  )
                  
                  
                  
  for i in range(0, len(urls)):
    download_url(urls[i], "data", class_name[i])

  print("----> Download Done!")
    
  step = ['train', 'validation', 'test']

  dire = r'data/'
  createDir(RESULT_FOLDER)

  max_length = 100000 # Maximum number of files (drawings) per class
  percen=[0.6, 0.3, 0.1] # Percentage of training, validation and testing

  begin = [0, int(max_length * percen[0]), int(max_length * (percen[0] + percen[1])) + 1]
  end = [int(max_length * (percen[0])), int(max_length * (percen[0] + percen[1])) + 1, max_length]

  for c in range(0, len(class_name)):
    print('Class ' + str(c+1) + ' out of ' + str(len(class_name)))
    filename = dire + str(class_name[c]) + '.npy'
    data = np.load(filename)

    for s in range(0, len(step)):
      dire_step = str(dire) + str(step[s])
      if not os.path.exists(dire_step):
        os.makedirs(dire_step)

      for i in range(begin[s], end[s]):
        dire_class = str(dire_step) + '/' + str(class_name[c])
        if not os.path.exists(dire_class):
          os.makedirs(dire_class)

        # Reshape the raw data into 28x28 images
        data_sample = data[i,:].reshape((28, 28))
        sample_name = class_name[c] + '_' + str(step[s]) + '_' + str(i)
        np.save(os.path.join(dire_class, sample_name), data_sample)

  0%|          | 0.00/126M [00:00<?, ?B/s]

126MB [00:01, 122MB/s]                           
  0%|          | 0.00/241M [00:00<?, ?B/s]

241MB [00:01, 137MB/s]                           
  1%|▏         | 1.47M/98.3M [00:00<00:06, 14.7MB/s]

98.3MB [00:00, 109MB/s]                             
  1%|          | 1.13M/181M [00:00<00:15, 11.3MB/s]

181MB [00:01, 109MB/s]                            
  1%|          | 827k/102M [00:00<00:12, 8.27MB/s]

...

107MB [00:01, 79.5MB/s]                            
  2%|▏         | 2.01M/98.8M [00:00<00:04, 20.1MB/s]

98.8MB [00:01, 88.5MB/s]                            
  1%|          | 975k/93.6M [00:00<00:09, 9.74MB/s]

93.6MB [00:00, 104MB/s]                             
  1%|          | 1.34M/113M [00:00<00:08, 13.3MB/s]

113MB [00:01, 107MB/s]                            


----> Download Done!
Class 1 out of 10
Class 2 out of 10
Class 3 out of 10
Class 4 out of 10
Class 5 out of 10
Class 6 out of 10
Class 7 out of 10
Class 8 out of 10
Class 9 out of 10
Class 10 out of 10


In [3]:
!ls data/test
!ls data/train
!ls data/validation

apple  banana  book  fork  key	ladder	pizza  stop_sign  tennis_racquet  wheel
apple  banana  book  fork  key	ladder	pizza  stop_sign  tennis_racquet  wheel
apple  banana  book  fork  key	ladder	pizza  stop_sign  tennis_racquet  wheel


In [0]:
LEARNING_RATE = 0.001        #.00001, 0.0001, 0.001, 0.1, 0.01
EPOCHS = 40


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(IMAGE_WIDTH * IMAGE_HEIGHT, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, N_CLASSES)
    def forward(self, x):
        x = x.view(-1, IMAGE_WIDTH * IMAGE_HEIGHT)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def load_sample(x):
	return np.load(x)

def train(epoch, net, train_loader, opt):
    net.train()
    correct = 0
    for j, item in enumerate(train_loader, 0):
      inputs, labels = item

      inputs = inputs.view(BATCH_SIZE, 1, IMAGE_WIDTH, IMAGE_HEIGHT).float()
      if CUDA:
          inputs = inputs.cuda()
          labels = labels.cuda()

      # Reset gradients
      opt.zero_grad()

      # Forward pass
      outputs = net(inputs)

      pred = outputs.data.max(1)[1]   # get the index of the max log-probability
      correct += pred.eq(labels.data).cpu().sum()
      accuracy = 100. * correct / len(train_loader.dataset)
      loss = criterion(outputs, labels)
      loss.backward()                 # calculate the gradients (backpropagation)
      optimizer.step()                # update the weights

      if j % 200 == 199:
        txt = '[%d, %5d] loss: %.3f' % (epoch, j + 1, loss.item())
        print(txt)

      

def validate(net, val_loader):
    net.eval()
    val_loss, correct, j = 0, 0, 0
    for inputs, labels in val_loader:

        inputs = inputs.view(BATCH_SIZE, 1, IMAGE_WIDTH, IMAGE_HEIGHT).float()
        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()

        outputs = net(inputs)
        val_loss += criterion(outputs, labels)
        pred = outputs.data.max(1)[1]
        correct += pred.eq(labels.data).cpu().sum()

        if j % 200 == 199:
          txt = '[%d, %5d] validation: %.3f' % (epoch, j + 1, loss.item())
          print(txt)
        j = j + 1

    val_loss /= len(val_loader)
    accuracy = 100. * correct / len(val_loader.dataset)

    print('\n Validation set  --  Average loss: {:.4f} \t Accuracy: {}/{} ({:.0f}%) \n'.format(
        val_loss, correct, len(val_loader.dataset), accuracy))
              
    return val_loss, accuracy
  
def plot(acc_vector, loss_vector):
    epochs = [i for i in range(1, EPOCHS + 1)]
    plt.plot(epochs, acc_vector)

    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

    plt.plot(epochs, loss_vector)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()


    
    
#  --  Start Training    
    
name = "MLP_model0" + "_Adam_" + str(BATCH_SIZE) + "_" + str(LEARNING_RATE).replace(".", "c") + "_" + str(EPOCHS)
#print(name)

train_dataset = datasets.DatasetFolder(TRAIN_FOLDER, extensions = ['.npy'], loader = load_sample)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0)

test_dataset = datasets.DatasetFolder(TEST_FOLDER, extensions = ['.npy'], loader = load_sample)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

val_dataset = datasets.DatasetFolder(VALID_FOLDER, extensions = ['.npy'], loader = load_sample)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)



net = MLP()

  
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = LEARNING_RATE)
if CUDA:
    net.cuda()

loss_vector = []
acc_vector = []
for ep in range(EPOCHS):  # epochs loop

    #train
    loss_info = train(ep, net, train_loader, optimizer)
    
    #validate
    #val_loss, accuracy = validate(net, val_loader)
    

    #loss_vector.append(val_loss)
    #acc_vector.append(accuracy)

    
# Plot train loss and validation accuracy vs epochs for each learning rate
#plot(acc_vector, loss_vector)
    
    

[0,   200] loss: 1.092
[0,   400] loss: 0.862
[0,   600] loss: 0.619
[0,   800] loss: 0.915
[0,  1000] loss: 0.738
[0,  1200] loss: 0.648
[0,  1400] loss: 0.490
[0,  1600] loss: 0.315
[0,  1800] loss: 0.831
[0,  2000] loss: 0.327
[0,  2200] loss: 0.682
[0,  2400] loss: 0.351
[0,  2600] loss: 0.419
[0,  2800] loss: 0.483
[0,  3000] loss: 0.498
